In [10]:
import pandas as pd
from pandas import json_normalize
import requests

In [11]:
BASE_URL = 'https://atriadev.atlassian.net/rest/api/2/'
TOKEN = 'HK0cCGHJ5YuxsU83gHxY38C3'
USER = 'ryan@atriadev.com'

In [12]:
def get_projects():
    url = BASE_URL + 'project/search'

    payload = {
        'expand': ['issueTypes', 'insight'],
    }

    r = requests.get(url, auth=(USER, TOKEN), params=payload)
    r = r.json()
    r = r['values']

    return r

In [13]:
def get_all_issues():
    '''Make the intial query'''
    proj_ids = '10015, 10010, 10000, 10022, 10012, 10019, 10017, 10014, 10013, 10020, 10021, 10023, 10024'

    def make_query(project_ids, max_results=100, start_at=0):

        url = BASE_URL + 'search'

        query = {
            'jql': 'project in ({})'.format(project_ids),
            'fields': [
                'id', 'self', 'key', 'summary', 'statuscategorychangedate',
                'issuetype', 'description', 'priority', 'project', 'status', 'created', 'updated', 'attachment'],
            'maxResults': max_results,
            'startAt': start_at,
        }

        r = requests.get(url, auth=(USER, TOKEN), params=query)
        r = r.json()

        return r

    results = make_query(proj_ids)

    total = results['total']
    issues = results['issues']

    if total > 100:
        paged_results = 0
        total_results = (total - 100)

        while total_results > 100:
            paged_results += 100
            results = make_query(proj_ids, start_at=paged_results)
            issues.extend(results['issues'])
            total_results -= 100
            print(len(issues))
        else:
            paged_results += 100
            results = make_query(proj_ids, max_results=total_results, start_at=paged_results)
            issues.extend(results['issues'])
            print(len(issues))

    return issues

In [14]:
issues = get_all_issues()
issues_df = json_normalize(data=issues, max_level=1, sep='_')
issues_df = issues_df.set_index("id", drop=False)
issues_df = issues_df.where(pd.notnull(issues_df), None)

200
289


In [15]:
db_issue_ids = [10002, 10003, 10013, 10015, 10016, 10017, 10018, 10019, 10020, 10021, 10022, 10024, 10059, 10060, 10061, 10062, 10063, 10064, 10065, 10066]


In [16]:
create_issues_df = issues_df[issues_df[issues_df.columns[1]].isin(db_issue_ids)]
create_issues_df

,expand,id,self,key,fields_statuscategorychangedate,fields_summary,fields_issuetype,fields_attachment,fields_created,fields_project,fields_description,fields_priority,fields_updated,fields_status
id,,,,,,,,,,,,,,


In [17]:
update_issues_df = issues_df[~issues_df[issues_df.columns[1]].isin(db_issue_ids)]
update_issues_df


,expand,id,self,key,fields_statuscategorychangedate,fields_summary,fields_issuetype,fields_attachment,fields_created,fields_project,fields_description,fields_priority,fields_updated,fields_status
id,,,,,,,,,,,,,,
10371,"operations,versionedRepresentations,editmeta,c...",10371,https://atriadev.atlassian.net/rest/api/2/issu...,TP-4,2021-08-10T12:20:16.120-0500,who knows all things,{'self': 'https://atriadev.atlassian.net/rest/...,[],2021-08-10T12:20:15.757-0500,{'self': 'https://atriadev.atlassian.net/rest/...,None,{'self': 'https://atriadev.atlassian.net/rest/...,2021-08-10T12:20:15.757-0500,{'self': 'https://atriadev.atlassian.net/rest/...
10370,"operations,versionedRepresentations,editmeta,c...",10370,https://atriadev.atlassian.net/rest/api/2/issu...,TP-3,2021-08-10T12:20:05.677-0500,are we alone in the universe,{'self': 'https://atriadev.atlassian.net/rest/...,[],2021-08-10T12:20:05.226-0500,{'self': 'https://atriadev.atlassian.net/rest/...,None,{'self': 'https://atriadev.atlassian.net/rest/...,2021-08-10T12:20:05.226-0500,{'self': 'https://atriadev.atlassian.net/rest/...
10322,"operations,versionedRepresentations,editmeta,c...",10322,https://atriadev.atlassian.net/rest/api/2/issu...,TP-2,2020-09-17T23:32:01.828-0500,fingerless downunder,{'self': 'https://atriadev.atlassian.net/rest/...,[],2020-09-17T23:32:01.327-0500,{'self': 'https://atriadev.atlassian.net/rest/...,might of had some issues with losing my finger...,{'self': 'https://atriadev.atlassian.net/rest/...,2020-09-18T22:16:21.229-0500,{'self': 'https://atriadev.atlassian.net/rest/...
10308,"operations,versionedRepresentations,editmeta,c...",10308,https://atriadev.atlassian.net/rest/api/2/issu...,TP-1,2020-03-26T19:45:05.146-0500,Have some fun with summaries,{'self': 'https://atriadev.atlassian.net/rest/...,[],2020-03-26T19:45:04.905-0500,{'self': 'https://atriadev.atlassian.net/rest/...,This is a description of a summary,{'self': 'https://atriadev.atlassian.net/rest/...,2020-03-26T19:45:04.905-0500,{'self': 'https://atriadev.atlassian.net/rest/...
10329,"operations,versionedRepresentations,editmeta,c...",10329,https://atriadev.atlassian.net/rest/api/2/issu...,NP-13,2021-02-12T16:15:49.089-0600,better summary,{'self': 'https://atriadev.atlassian.net/rest/...,[],2021-02-12T16:15:48.802-0600,{'self': 'https://atriadev.atlassian.net/rest/...,None,{'self': 'https://atriadev.atlassian.net/rest/...,2021-02-12T16:15:48.802-0600,{'self': 'https://atriadev.atlassian.net/rest/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10120,"operations,versionedRepresentations,editmeta,c...",10120,https://atriadev.atlassian.net/rest/api/2/issu...,ACW-5,2019-11-11T09:57:22.004-0600,Admin area - Show info that admin is logged in...,{'self': 'https://atriadev.atlassian.net/rest/...,[],2019-11-10T11:13:39.064-0600,{'self': 'https://atriadev.atlassian.net/rest/...,There should show something so the admin knows...,{'self': 'https://atriadev.atlassian.net/rest/...,2019-11-11T09:41:45.872-0600,{'self': 'https://atriadev.atlassian.net/rest/...
10119,"operations,versionedRepresentations,editmeta,c...",10119,https://atriadev.atlassian.net/rest/api/2/issu...,ACW-4,2019-11-11T10:36:57.977-0600,admin area - Issues with admin mimicking user ...,{'self': 'https://atriadev.atlassian.net/rest/...,[],2019-11-10T11:10:22.571-0600,{'self': 'https://atriadev.atlassian.net/rest/...,As an admin I mimicked a user and then hit log...,{'self': 'https://atriadev.atlassian.net/rest/...,2019-11-11T10:36:57.977-0600,{'self': 'https://atriadev.atlassian.net/rest/...
10118,"operations,versionedRepresentations,editmeta,c...",10118,https://atriadev.atlassian.net/rest/api/2/issu...,ACW-3,2019-11-11T10:36:50.350-0600,Admin area - Add hover text to Action buttons,{'self': 'https://atriadev.atlassian.net/rest/...,[],2019-11-10T11:09:21.797-0600,{'self': 'https://atriadev.atlassian.net/rest/...,None,{'self': 'https://atriadev.atlassian.net/rest/...,2019-11-11T10:36:50.349-0600,{'self': 'https://atriadev.atlassia

In [18]:
def issue_assignment(df):
    records = df.to_dict('records')
    instances = [Issue(
        id=int(record['id']),
        key=record['key'],
        url=record['self'],
        summary=record['fields_summary'],
        description=record['fields_description'],
        status_change_date=record['fields_statuscategorychangedate'],
        created_at=record['fields_created'],
        updated_at=record['fields_updated'],
        status_id=record['fields_status']['id'],
        status_name=record['fields_status']['name'],
        priority_id=record['fields_priority']['id'],
        priority_name=record['fields_priority']['name'],

        project=Project.objects.get(id=int(record['fields_project']['id'])),
        issue_type=IssueTypes.objects.get(id=int(record['fields_issuetype']['id'])),

    ) for record in records]

    return instances

In [19]:
issue_fields = [
        'key',
        'url',
        'summary',
        'description',
        'status_change_date',
        'created_at',
        'updated_at',
        'status_id',
        'status_name',
        'priority_id',
        'priority_name',
        'project',
        'issue_type',
    ]

In [20]:
if create_issues_df.empty:
    print('create issues empty')



create issues empty


In [21]:
def get_project_issues(project_id):
    '''Make the intial query'''
#     proj_ids = '10015, 10010, 10000, 10022, 10012, 10019, 10017, 10014, 10013, 10020, 10021, 10023, 10024'

    def make_query(project_ids, max_results=100, start_at=0):

        url = BASE_URL + 'search'

        query = {
            'jql': 'project in ({})'.format(project_ids),
            'fields': [
                'id', 'self', 'key', 'summary', 'statuscategorychangedate',
                'issuetype', 'description', 'priority', 'project', 'status', 'created', 'updated', 'attachment'],
            'maxResults': max_results,
            'startAt': start_at,
        }

        r = requests.get(url, auth=(USER, TOKEN), params=query)
        r = r.json()

        return r

    results = make_query(project_id)

    total = results['total']
    issues = results['issues']

    if total > 100:
        paged_results = 0
        total_results = (total - 100)

        while total_results > 100:
            paged_results += 100
            results = make_query(proj_ids, start_at=paged_results)
            issues.extend(results['issues'])
            total_results -= 100
            print(len(issues))
        else:
            paged_results += 100
            results = make_query(proj_ids, max_results=total_results, start_at=paged_results)
            issues.extend(results['issues'])
            print(len(issues))

    return issues

In [22]:
proj_issues = get_project_issues(10024)

In [23]:
proj_issues

[{'expand': 'operations,versionedRepresentations,editmeta,changelog,renderedFields',
  'id': '10371',
  'self': 'https://atriadev.atlassian.net/rest/api/2/issue/10371',
  'key': 'TP-4',
  'fields': {'summary': 'who knows all things',
   'statuscategorychangedate': '2021-08-10T12:20:16.120-0500',
   'issuetype': {'self': 'https://atriadev.atlassian.net/rest/api/2/issuetype/10075',
    'id': '10075',
    'description': 'Tasks track small, distinct pieces of work.',
    'iconUrl': 'https://atriadev.atlassian.net/secure/viewavatar?size=medium&avatarId=10318&avatarType=issuetype',
    'name': 'Task',
    'subtask': False,
    'avatarId': 10318,
    'entityId': 'c84e7a75-e589-419e-8f0b-6ff2e665f180',
    'hierarchyLevel': 0},
   'attachment': [],
   'created': '2021-08-10T12:20:15.757-0500',
   'project': {'self': 'https://atriadev.atlassian.net/rest/api/2/project/10024',
    'id': '10024',
    'key': 'TP',
    'name': 'test_project',
    'projectTypeKey': 'software',
    'simplified': True,

In [24]:
proj_issues_df = json_normalize(data=proj_issues, max_level=1, sep='_')
proj_issues_df = proj_issues_df.set_index("id", drop=False)
proj_issues_df = proj_issues_df.where(pd.notnull(issues_df), None)

In [25]:
# ids need to be strings
proj_issue_ids = ['10322','10308']

In [26]:
# querys dfs id field and compares with proj_issue_ids
update_issues = proj_issues_df.query('id in @proj_issue_ids')

In [31]:
new_issues = proj_issues_df.query('id not in @proj_issue_ids')
new_issues.iloc[:, 1]

id
10371    10371
10370    10370
Name: id, dtype: object